In [2]:
%pip install nltk

import os
import math
from collections import defaultdict
import re
import nltk
nltk.download('all')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: C:\Users\Ishan\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\Ishan\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\Ishan\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\Ishan\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     C:\Users\Ishan\AppData\Roaming\nltk_data...
[nltk_data]  

In [2]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Ishan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ishan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Ishan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Ishan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [7]:
def preprocess(text):
    # Lowercase
    text = text.lower()
    
    # Remove special characters
    text = re.sub(r'[^\w\s]', '', text)
    
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    
    # Tokenize
    text = word_tokenize(text)
    
    # Remove stopwords and lemmatize
    stop_words = set(stopwords.words('english'))
    text = [word for word in text if word not in stop_words]
    lemmatizer = WordNetLemmatizer()
    text = [lemmatizer.lemmatize(word) for word in text]
    
    # Stemming
    stemmer = nltk.stem.PorterStemmer()
    text = [stemmer.stem(word) for word in text]
    
    return text

In [9]:
def read_documents(directory):
    documents = defaultdict(list)
    try:
        for filename in os.listdir(directory):
            with open(os.path.join(directory, filename), 'r', encoding = 'utf-8') as file:
                text = file.read()
                tokenized = preprocess(text)
                documents[filename] = tokenized
    except Exception as e:
        print('An error occurred:', e)
    return documents

print(read_documents('Corpus'))

defaultdict(<class 'list'>, {'Adobe.txt': ['adob', 'compani', 'found', 'john', 'warnock', 'charl', 'geschk', 'employ', 'xerox', 'corpor', 'palo', 'alto', 'california', 'research', 'center', 'parc', 'two', 'comput', 'scientist', 'develop', 'program', 'languag', 'special', 'design', 'describ', 'precis', 'posit', 'shape', 'size', 'object', 'computergener', 'page', 'page', 'descript', 'languag', 'later', 'known', 'postscript', 'describ', 'object', 'letter', 'graphic', 'mathemat', 'term', 'without', 'refer', 'specif', 'comput', 'printer', 'devic', 'capabl', 'interpret', 'languag', 'would', 'abl', 'gener', 'represent', 'page', 'resolut', 'devic', 'support', 'xerox', 'declin', 'bring', 'technolog', 'market', 'warnock', 'geschk', 'form', 'compani', 'name', 'creek', 'near', 'home', 'appl', 'comput', 'inc', 'appl', 'inc', 'acquir', 'percent', 'adob', 'becam', 'first', 'license', 'postscript', 'appl', 'introduc', 'first', 'macintoshcompat', 'postscript', 'printer', 'laserwrit', 'base', 'laserprin

In [22]:
# Computing the term frequency

def term_frequency(documents):
    tf = defaultdict(dict)
    for doc_id, doc in documents.items():
        for term in doc:
            if term in tf[doc_id]:
                tf[doc_id][term] += 1
            else:
                tf[doc_id][term] = 1
    return tf

# Weighted term frequency

def weighted_term_frequency(tf):
    weighted_tf = defaultdict(dict)
    for doc_id, doc in tf.items():
        for term, freq in doc.items():
            weighted_tf[doc_id][term] = 1 + math.log10(freq)
    return weighted_tf

In [23]:
# Compute inverse-document frequency

def inverse_document_frequency(documents):
    idf = defaultdict(int)
    for doc_id, doc in documents.items():
        for term in set(doc):
            idf[term] += 1
    for term, freq in idf.items():
        idf[term] = math.log10(len(documents) / freq)
    return idf

# Compute tf-idf
def tf_idf(documents):
    tf = term_frequency(documents)
    weighted_tf = weighted_term_frequency(tf)
    idf = inverse_document_frequency(documents)
    tf_idf = defaultdict(dict)
    for doc_id, doc in documents.items():
        for term in doc:
            tf_idf[doc_id][term] = weighted_tf[doc_id][term] * idf[term]
    return tf_idf

tf_idf(read_documents('Corpus'))

defaultdict(dict,
            {'Adobe.txt': {'adob': 3.7452377903099547,
              'compani': 0.24122862537672854,
              'found': 0.2703611758975292,
              'john': 1.3117538610557542,
              'warnock': 2.0982801741150165,
              'charl': 1.3117538610557542,
              'geschk': 2.0982801741150165,
              'employ': 0.6585413472804106,
              'xerox': 2.0982801741150165,
              'corpor': 0.612523503197519,
              'palo': 1.6127838567197355,
              'alto': 1.6127838567197355,
              'california': 1.0107238653917732,
              'research': 0.5713911715615104,
              'center': 0.9138138523837167,
              'parc': 1.6127838567197355,
              'two': 0.43458338221205695,
              'comput': 0.6092426477569958,
              'scientist': 1.135662602000073,
              'develop': 0.39640796294192815,
              'program': 0.7777241267761748,
              'languag': 1.1369721547816145,
  